# Moviepy

In [1]:
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import numpy as np
from PIL import ImageFont, ImageDraw, Image

In [2]:
# 포즈 감지 모델 초기화
mp_pose = mp.solutions.pose
pose_video = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.7,
                          min_tracking_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

In [3]:
# 포즈 검출 함수
def detectPose(image_pose, pose, draw=False, display=False):
    
    original_image = image_pose.copy()
    
    image_in_RGB = cv2.cvtColor(image_pose, cv2.COLOR_BGR2RGB)
    
    resultant = pose.process(image_in_RGB)

    if resultant.pose_landmarks and draw:    

        mp_drawing.draw_landmarks(image=original_image, landmark_list=resultant.pose_landmarks,
                                  connections=mp_pose.POSE_CONNECTIONS,
                                  landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255,255,255),
                                                                               thickness=3, circle_radius=3),
                                  connection_drawing_spec=mp_drawing.DrawingSpec(color=(49,125,237),
                                                                               thickness=2, circle_radius=2))

    if display:
            
            plt.figure(figsize=[22,22])
            plt.subplot(121);plt.imshow(image_pose[:,:,::-1]);plt.title("Input Image");plt.axis('off');
            plt.subplot(122);plt.imshow(original_image[:,:,::-1]);plt.title("Pose detected Image");plt.axis('off');

    else:
        
        return original_image, resultant

In [4]:
def norm(data):
    data = np.array(data)
    x = data.T[0]
    y = data.T[1]
    z = data.T[2]
    x_norm = (x - min(x)) / (max(x) - min(x))
    y_norm = (y - min(y)) / (max(y) - min(y))
    z_norm = (z - min(z)) / (max(z) - min(z))
    
    return (x_norm.tolist(), y_norm.tolist(), z_norm.tolist())

In [5]:
def link_vector(land):
    link_keypoint = [(0, 1),
        (1, 3),
        (3,	5),
        (5,	7),
        (5,	9),
        (5,	11),
        (1, 13),
        (13, 15),
        (15, 17),
        (17, 19),
        (17, 21),
        (0, 2),
        (2, 4),
        (4, 6),
        (6, 8),
        (6, 10),
        (6, 12),
        (2, 14),
        (14, 16),
        (16, 18),
        (18, 20),
        (18, 22)]    
    
    a = []
    for link in link_keypoint:
        x = land[0][link[0]] - land[0][link[1]]
        y = land[1][link[0]] - land[1][link[1]]
        z = land[2][link[0]] - land[2][link[1]]
        a.append((x, y, z))
    return np.array(a)

In [6]:
def angle_vector(land):
    
    import math
    
    angle_keypoint=[
        (0, 1, 3),
        (1, 3, 5),
        (3, 5, 9),
        (1, 13, 15),
        (13, 15, 17),
        (15, 17, 19),
        (15, 17, 21),
        (0, 2, 4),
        (2, 4, 6),
        (4, 6, 10),
        (2, 14, 16),
        (14, 16, 18),
        (16, 18, 20),
        (16, 18, 22)]
    
    a = []
    for angle in angle_keypoint:
        x = np.array([land[0][angle[0]] - land[0][angle[1]], land[1][angle[0]] - land[1][angle[1]], land[2][angle[0]] - land[2][angle[1]]])
        y = np.array([land[0][angle[1]] - land[0][angle[2]], land[1][angle[1]] - land[1][angle[2]], land[2][angle[1]] - land[2][angle[2]]])
        
        분자 = np.dot(x, y)
        분모 = np.sqrt(x.dot(x)) * np.sqrt(x.dot(x))
        try:
            a.append(math.acos(분자 / 분모))
        except:
            a.append(0)
    return (a)

In [7]:
def pose_feature(link, angle):
    산술평균_링크 = [np.mean(link.T[0]), np.mean(link.T[1]), np.mean(link.T[2])]
    표준편차_링크 = [np.std(link.T[0]), np.std(link.T[1]), np.std(link.T[2])]
    제곱평균_링크 = [np.mean(link.T[0]**2), np.mean(link.T[1]**2), np.mean(link.T[2]**2)]
    
    산술평균_앵글 = np.mean(angle)
    표준편차_앵글 = np.std(angle)
    제곱평균_앵글 = np.mean(angle)
    
    return(산술평균_링크+표준편차_링크+제곱평균_링크+[산술평균_앵글]+[표준편차_앵글]+[제곱평균_앵글])

In [8]:
import joblib
model = joblib.load('./RandomForestFinal.pkl')

In [9]:
def get_sim(target, user):
    target_landmarks = target.pose_world_landmarks.landmark
    target_lm = [(i.x, i.y, i.z) for num, i in enumerate(target_landmarks) if num not in range(1, 11)]
    target_norm = norm(target_lm)
    target_link_vector = link_vector(target_norm)
    target_angle_vector = angle_vector(target_norm)
    
    user_landmarks = user.pose_world_landmarks.landmark
    user_lm = [(i.x, i.y, i.z) for num, i in enumerate(user_landmarks) if num not in range(1, 11)]
    user_norm = norm(user_lm)
    user_link_vector = link_vector(user_norm)
    user_angle_vector = angle_vector(user_norm)
    
    link_diff = np.array(target_link_vector) - user_link_vector
    angle_diff = np.array(target_angle_vector) - user_angle_vector
    feature = pose_feature(link_diff, angle_diff); feature = np.array(feature).reshape(1, -1)
    similarity = model.predict_proba(feature)[0][1]
    
    return similarity

## BTS - Dynamite

In [10]:
from moviepy.editor import *
video_clip = VideoFileClip('./static/dance/BTS - Dynamite.mp4')
du = video_clip.duration
video_clip2 = VideoFileClip('./static/user_dance/Dynamite user.mp4').subclip(0.2, du+0.2)

In [11]:
audioclip = video_clip.audio
audioclip.write_audiofile('./target.mp3')

MoviePy - Writing audio in ./target.mp3


MoviePy - Done.


In [14]:
# 사용자 영상만
w = int(video_clip2.w)
h = int(video_clip2.h)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, 30, (w, h))  # out 1/0.03 -> 1 ///

ss = []
text = ''

for num, i in enumerate(np.arange(0, du, 0.03)):  # 원래는 0.03
   
    img = video_clip.get_frame(i)
    img2 = video_clip2.get_frame(i)
    
    img_user = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
    
    if (num+1) % 20 == 0:
        ss_mean = round(np.mean([i*100 for i in ss[num-18:] if i is not None]))
        score = ('BAD' if ss_mean <= 30 else 'MISS' if ss_mean <= 40 else 'GOOD' if ss_mean <= 50 else 'VERY GOOD' if ss_mean <= 60 else 'PERPECT')
        text = f'{score}'

    try:
        result = detectPose(img, pose_video)[1]
        result2 = detectPose(img2, pose_video)[1]
        
        similarity = get_sim(result, result2)
        ss.append(similarity)
        

        
        cv2.putText(img_user,  text, (500, 50), cv2.FONT_HERSHEY_DUPLEX, 2, (199, 114, 255), 2, cv2.LINE_AA)        # BGR
        cv2.imshow("target", img_user)
        
    except:
        ss.append(None)
        
        cv2.putText(img_user,  text, (500, 50), cv2.FONT_HERSHEY_DUPLEX, 2, (199, 114, 255), 2, cv2.LINE_AA)        # BGR
        cv2.imshow("target", img_user)
        
        
    out.write(img_user) #프레임 쓰기
    
    if cv2.waitKey(1) & 0xFF == 27:
        break

cv2.destroyAllWindows()
out.release()

..........................................................................................................................................................................................................................................................................................................................................................................................

In [42]:
# 사용자 영상만
w = int(video_clip2.w)
h = int(video_clip2.h)
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
out = cv2.VideoWriter('0.avi', fourcc, 1, (w, h))

ss = []
text = ''

for num, i in enumerate(np.arange(0, du, 1)):
   
    img = video_clip.get_frame(i)
    img2 = video_clip2.get_frame(i)
    
    img_user = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
    
    if (num+1) % 20 == 0:
        ss_mean = round(np.mean([i*100 for i in ss[num-18:] if i is not None]))
        score = ('BAD' if ss_mean <= 30 else 'MISS' if ss_mean <= 40 else 'GOOD' if ss_mean <= 50 else 'VERY GOOD' if ss_mean <= 60 else 'PERPECT')
        text = f'{score}'

    try:
        result = detectPose(img, pose_video)[1]
        result2 = detectPose(img2, pose_video)[1]
        
        similarity = get_sim(result, result2)
        ss.append(similarity)

        
        cv2.putText(img_user,  text, (500, 50), cv2.FONT_HERSHEY_DUPLEX, 2, (199, 114, 255), 2, cv2.LINE_AA)        # BGR
        cv2.imshow("target", img_user)
        
    except:
        ss.append(None)
        
        cv2.putText(img_user,  text, (500, 50), cv2.FONT_HERSHEY_DUPLEX, 2, (199, 114, 255), 2, cv2.LINE_AA)        # BGR
        cv2.imshow("target", img_user)

    out.write(img_user)
    
    if cv2.waitKey(1) & 0xFF == 27:
        break

cv2.destroyAllWindows()
out.release()

In [40]:
# target + user 영상 넓이를 640으로 맞추기
import imutils

w = 640
h = 720
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
out = cv2.VideoWriter('output.avi', fourcc, 1/0.03, (w, h))

ss = []
text = ''
sc = []

for num, i in enumerate(np.arange(0, du, 0.03)):
   
    img = video_clip.get_frame(i)
    img2 = video_clip2.get_frame(i)
    
    img_taget = imutils.resize(img, width=640)
    img_user = imutils.resize(img2, width=640)
    
    if (num+1) % 20 == 0:
        ss_mean = round(np.mean([i*100 for i in ss[num-18:] if i is not None]))
        sc.append(ss_mean)
        score = ('BAD' if ss_mean <= 30 else 'MISS' if ss_mean <= 40 else 'GOOD' if ss_mean <= 50 else 'VERY GOOD' if ss_mean <= 60 else 'PERPECT')
        text = f'{score}'

    try:
        result = detectPose(img, pose_video)[1]
        result2 = detectPose(img2, pose_video)[1]
        
        similarity = get_sim(result, result2)
        ss.append(similarity)

        numpy_vertical = np.vstack((img_taget, img_user))
        
        ver_cv = cv2.cvtColor(numpy_vertical, cv2.COLOR_BGR2RGB)
        
        cv2.putText(ver_cv,  text, (10, 380), cv2.FONT_HERSHEY_DUPLEX, 2, (199, 114, 255), 2, cv2.LINE_AA)        # BGR
        cv2.imshow("target", ver_cv)
        
    except:
        numpy_vertical = np.vstack((img_taget, img_user))
        ss.append(None)
        
        ver_cv = cv2.cvtColor(numpy_vertical, cv2.COLOR_BGR2RGB)
        cv2.putText(ver_cv,  text, (10, 380), cv2.FONT_HERSHEY_DUPLEX, 2, (199, 114, 255), 2, cv2.LINE_AA)        # BGR
        cv2.imshow("target", ver_cv)

    out.write(ver_cv) #프레임 쓰기
    
    if cv2.waitKey(1) & 0xFF == 27:
        break

# 점수 프레임 만들기
font = ImageFont.truetype("C:/Users/min/AppData/Local/Microsoft/Windows/Fonts/NanumSquareOTF.otf", 40)

img = np.full(ver_cv.shape, (255, 255, 255), np.uint8)
img = Image.fromarray(img)

draw = ImageDraw.Draw(img)

total_score = round(np.mean(sc))
text = f"{total_score} 점"
text_score = ('FAIL' if total_score <= 30 else 'BAD' if total_score <= 40 else 'OK' if total_score <= 50 else 'GOOD' if total_score <= 60 else 'GREAT')

draw.text((270, 100),  text, font=font, fill=(0, 0, 0))
draw.text((270, 200),  text_score, font=font, fill=(0, 0, 0))

img = np.array(img)

cv2.imshow("target", img)
for i in range(0, 100):
    out.write(img) #프레임 쓰기
cv2.waitKey(5000)

cv2.destroyAllWindows()
out.release()


OSError: cannot open resource

In [41]:
out.release()

In [ ]:
# 영상 opencv로 재생되는지 확인

w = 640
h = 720
cap = cv2.VideoCapture('output.avi')
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH) # 또는 cap.get(3)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT) # 또는 cap.get(4)
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'DIVX') # 코덱 정의
out = cv2.VideoWriter('otter_out.avi', fourcc, fps, (int(width), int(height))) # VideoWriter 객체 정의

while cap.isOpened():
    
    ret, frame = cap.read()
    
    if not ret:
        print("영상을 재생할수 없습니다.")
        break
    frame = cv2.cvtColor(frame, cv2.COLORBGR2RGB)
    cv2.imshow('output', frame)
    if cv2.waitKey(42) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [174]:
out = cv2.VideoWriter('0.avi', fourcc, 1/0.03, (w, h))

# 점수 프레임 만들기
total_score = round(np.mean(sc))
text_score = ('F' if total_score <= 30 else 'D' if total_score <= 40 else 'C' if total_score <= 50 else 'B' if total_score <= 60 else 'A')
score_img = cv2.imread(f'./score/{text_score}.PNG',1)
score_img = imutils.resize(score_img, width=640)
img_st = round((score_img.shape[0]-ver_cv.shape[0])/2)
score_img = score_img[img_st:img_st+img_st + ver_cv.shape[0], :, :]

for i in range(0, 100):
    out.write(img) #프레임 쓰기

In [108]:
all = []
for i in sc:
    k = 'FAIL' if i <= 30 else 'BAD' if i <= 40 else 'OK' if i <= 50 else 'GOOD' if i <= 60 else 'GREAT'
    all.append(k)

In [105]:
total_score = round(np.mean(sc))
print("최종 점수: ", total_score)
print('FAIL' if total_score <= 30 else 'BAD' if total_score <= 40 else 'OK' if total_score <= 50 else 'GOOD' if total_score <= 60 else 'GREAT')

최종 점수:  38
BAD


In [ ]:
from moviepy.editor import *
videoclip = VideoFileClip("output.avi")
audioclip = AudioFileClip("target.mp3")

videoclip.audio = audioclip
videoclip.write_videofile("new video.mp4")

In [137]:
import cv2
img_color = cv2.imread('./score/A.PNG',1) #cv2.IMREAD_COLOR
img = imutils.resize(img_color, width=640)
img = img[208:208+720, :]

cv2.imshow('color', img)
cv2.waitKey(0) 
cv2.destroyAllWindows()

- Bad Pose

In [141]:
k = [i for i in ss if i is not None]
start = np.arange(0, du, 0.03)[k.index(min(k))]
end = min(start+20, du)

In [ ]:
bad_pose = VideoFileClip('./new video.mp4').subclip(start, end)
bad_pose.write_videofile("./badpose2.mp4")

## I-DLE - TOMBOY

In [11]:
from moviepy.editor import *
video_clip = VideoFileClip('./영상/I-DLE TOMBOY.mp4').subclip(4.3, 83.48)
video_clip2 = VideoFileClip('./영상/I-DLE TOMBOY_user.mp4').subclip(0.7, 74.9)

In [12]:
# 싱크 맞는지 확인
for i in np.arange(0, 84.2, 0.03):
   
    img = video_clip.get_frame(i)
    img2 = video_clip2.get_frame(i)
    
    img_taget = cv2.resize(img, (0, 0), fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
    img_user = cv2.resize(img2, (0, 0), fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)

    numpy_vertical = np.vstack((img_taget, img_user))
    numpy_vertical_concat = np.concatenate((img_taget, img_user), axis=0)
    
    ver_cv = cv2.cvtColor(numpy_vertical, cv2.COLOR_BGR2RGB)
    cv2.imshow("target", ver_cv)
        
    # out.write(ver_cv) #프레임 쓰기
    
    if cv2.waitKey(1) & 0xFF == 27:
        break

cv2.destroyAllWindows()

# 웹캠 녹화

In [10]:
import cv2
import time
cap = cv2.VideoCapture(cv2.CAP_DSHOW+0)
cap.set(3, 720) # 윈도우 크기
cap.set(4, 1080)
fc = 30.0
codec = cv2.VideoWriter_fourcc('D', 'I', 'V', 'X')
count = 99
while(cap.isOpened()):
    
    if count != time.strftime('%H',time.localtime(time.time())): # 시간이 바뀌면 영상파일을 새로 만든다. (시간으로 감지)
        
        count = time.strftime('%H',time.localtime(time.time()))
        print('시간 변경 감지')
        
        out = cv2.VideoWriter(time.strftime('%Y-%m-%d %H시 %M분',time.localtime(time.time()))+'.avi', codec, fc, (int(cap.get(3)), int(cap.get(4))))
        print('파일 생성:',time.strftime('%Y-%m-%d %H시 %M분',time.localtime(time.time()))+'.avi')
    
    ret, frame = cap.read()
    #frame = cv2.flip(frame,1) # 화면 반전 0: 상하, 1: 좌우
    # 시간 텍스트 출력
    cv2.putText(frame, text=time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())), org=(30, 450), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0,255,0), thickness=2)
    
    if ret==True:
        cv2.imshow('Record&Save', frame)
        out.write(frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
    
cap.release()
out.release()
cv2.destroyAllWindows()

시간 변경 감지
파일 생성: 2022-07-20 13시 45분.avi


In [28]:
import cv2
import time


# 이미지에 텍스트를 출력하는 함수
def draw_text(img, text, x, y):
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    font_thickness = 2
    text_color=(255, 0, 0)
    text_color_bg=(0, 0, 0)

    text_size, _ = cv2.getTextSize(text, font, font_scale, font_thickness)
    text_w, text_h = text_size
    offset = 5

    cv2.rectangle(img, (x - offset, y - offset), (x + text_w + offset, y + text_h + offset), text_color_bg, -1)
    cv2.putText(img, text, (x, y + text_h + font_scale - 1), font, font_scale, text_color, font_thickness)


# 웹캠 연결
cap = cv2.VideoCapture(0)


# 웹캠에서 fps 값 획득
fps = cap.get(cv2.CAP_PROP_FPS)
print('fps', fps)

if fps == 0.0:
    fps = 30.0

time_per_frame_video = 1/fps
last_time = time.perf_counter()


while True:

    # 웹캠에서 이미지 읽어옴
    ret,img_color = cap.read()

    if ret == False:
        print('웹캠에서 영상을 읽을 수 없습니다.')
        break

    # fsp 계산
    time_per_frame = time.perf_counter() - last_time
    time_sleep_frame = max(0, time_per_frame_video - time_per_frame)
    time.sleep(time_sleep_frame)

    real_fps = 1/(time.perf_counter()-last_time)
    last_time = time.perf_counter()


    x = 30
    y = 50
    text = '%.2f fps' % real_fps

    # 이미지의 (x, y)에 텍스트 출력
    draw_text(img_color, text, x, y)
    cv2.imshow("Color", img_color)


    # ESC키 누르면 중지
    if cv2.waitKey(1)&0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

fps 30.0


In [48]:
import cv2
import sys
import time
# 노트북 웹캠에서 받아오는 영상을 저장하기

# 기본 카메라 객체 생성
cap = cv2.VideoCapture(0)

# 열렸는지 확인
if not cap.isOpened():
    print("Camera open failed!")
    sys.exit()

# 웹캠의 속성 값을 받아오기
# 정수 형태로 변환하기 위해 round
w = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS) # 카메라에 따라 값이 정상적, 비정상적

if fps ==0 :
    fps=30

# fourcc 값 받아오기, *는 문자를 풀어쓰는 방식, *'DIVX' == 'D', 'I', 'V', 'X'
fourcc = cv2.VideoWriter_fourcc(*'DIVX')

# 1프레임과 다음 프레임 사이의 간격 설정
delay = round(1000/fps)

# 웹캠으로 찰영한 영상을 저장하기
# cv2.VideoWriter 객체 생성, 기존에 받아온 속성값 입력
out = cv2.VideoWriter('1.avi', fourcc, fps, (w, h))

begins = []
# 제대로 열렸는지 확인
if not out.isOpened():
    print('File open failed!')
    cap.release()
    sys.exit()
    
# 프레임을 받아와서 저장하기
while True:                 # 무한 루프
    ret, frame = cap.read() # 카메라의 ret, frame 값 받아오기

    if not ret:             #ret이 False면 중지
        break
    
    cv2.putText(frame, text=time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())), org=(30, 450), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0,255,0), thickness=2)
    cv2.imshow('frame', frame)
    begins.append(time.time())
    out.write(frame) # 영상 데이터만 저장. 소리는 X
    
    # print(cap.get(cv2.CAP_PROP_FPS))
    if cv2.waitKey(delay) == 27: # esc를 누르면 강제 종료
        end = time.time()
        break

cap.release()
out.release()
cv2.destroyAllWindows()